# SUBIR MULTAS

# IMPORTANDO LIBRERIAS

In [1]:
import pandas as pd
import MySQLdb

# CONEXION A LA BASE DE DATOS

In [2]:
db = MySQLdb.connect(host="172.18.55.6",    # tu host, usualmente localhost
                     user="comandato",         # tu usuario
                     passwd="comandato123",  # tu password
                     db="SISTEMECUADOR_ATM")    # el nombre de la base de datos
cur= db.cursor()

# FECHAS DE COMPARACION

### FECHA ACTUAL

In [3]:
Anio_act='2019'
Mes_act='01'
Dia_act='24'

### FECHA ANTERIOR

In [4]:
Anio_ant='2019'
Mes_ant='01'
Dia_ant='23'

# CARACTERISTICA DE LOS ARCHIVOS

### ACTUALES

In [5]:
corr_act={
    'ruta':'..\..\..\..\BASES DATATOOLS\ASIGNACION\\2019\\ENERO\Sistema de cobro_'+Dia_act+'_'+Mes_act+'_'+Anio_act+'.xlsx',
    'hoja':'Asignacion',
    'cartera':'CORRIENTE',
    'identificador':'IDENTIFICACION_DEUDOR'
}

men_act={
    'ruta':'..\..\..\..\BASES DATATOOLS\ASIGNACION\\2019\\ENERO\Sistema de cobro_'+Dia_act+'_'+Mes_act+'_'+Anio_act+'.xlsx',
    'hoja':'Menores',
    'cartera':'MENOR',
    'identificador':'IDENTIFICACION_DEUDOR'
}

conv_act={
    'ruta':'..\..\..\..\BASES DATATOOLS\ASIGNACION\\2019\\ENERO\CONVENIOS_SISTEM_'+Dia_act+'_'+Mes_act+'_'+Anio_act+'.xlsx',
    'hoja':'Hoja1',
    'cartera':'CONVENIOS',
    'identificador':'NUMERO_IDENTIFICACION'
}

dem_act={
    'ruta':'..\..\..\..\BASES DATATOOLS\ASIGNACION\\2019\\ENERO\Sistema de cobro_'+Dia_act+'_'+Mes_act+'_'+Anio_act+'.xlsx',
    'hoja':'Demografico',
    'cartera':'DEMOGRAFICO',
    'identificador':'IDENTIFICACION_DEUDOR'
}

###  ANTERIORES

In [6]:
corr_ant={
    'ruta':'..\..\..\..\BASES DATATOOLS\ASIGNACION\\2019\\ENERO\Sistema de cobro_'+Dia_ant+'_'+Mes_ant+'_'+Anio_ant+'.xlsx',
    'hoja':'Asignacion',
    'cartera':'CORRIENTE',
    'identificador':'IDENTIFICACION_DEUDOR'
}

men_ant={
    'ruta':'..\..\..\..\BASES DATATOOLS\ASIGNACION\\2019\\ENERO\Sistema de cobro_'+Dia_ant+'_'+Mes_ant+'_'+Anio_ant+'.xlsx',
    'hoja':'Menores',
    'cartera':'MENOR',
    'identificador':'IDENTIFICACION_DEUDOR'
}
conv_ant={
    'ruta':'..\..\..\..\BASES DATATOOLS\ASIGNACION\\2019\\ENERO\CONVENIOS_SISTEM_'+Dia_ant+'_'+Mes_ant+'_'+Anio_ant+'.xlsx',
    'hoja':'Hoja1',
    'cartera':'CONVENIOS',
    'identificador':'NUMERO_IDENTIFICACION'
}
dem_ant={
    'ruta':'..\..\..\..\BASES DATATOOLS\ASIGNACION\\2019\\ENERO\Sistema de cobro_'+Dia_ant+'_'+Mes_ant+'_'+Anio_ant+'.xlsx',
    'hoja':'Demografico',
    'cartera':'DEMOGRAFICO',
    'identificador':'IDENTIFICACION_DEUDOR'
}

# LEER ARCHIVOS

In [7]:
arch_corr_act=pd.read_excel(open(corr_act['ruta'],'rb'), sheet_name=corr_act['hoja'], dtype={corr_act['identificador']: str})
arch_corr_ant=pd.read_excel(open(corr_ant['ruta'],'rb'), sheet_name=corr_ant['hoja'], dtype={corr_ant['identificador']: str})
arch_menr_act=pd.read_excel(open(men_act['ruta'],'rb'), sheet_name=men_act['hoja'], dtype={men_act['identificador']: str})
arch_menr_ant=pd.read_excel(open(men_ant['ruta'],'rb'), sheet_name=men_ant['hoja'], dtype={men_ant['identificador']: str})
# arch_conv_act=pd.read_excel(open(conv_act['ruta'],'rb'), sheet_name=conv_act['hoja'], dtype={conv_act['identificador']: str})
# arch_conv_ant=pd.read_excel(open(conv_ant['ruta'],'rb'), sheet_name=conv_ant['hoja'], dtype={conv_ant['identificador']: str})

## COLUMNA DE COMPARACION

In [8]:
base_actual = arch_corr_act
base_anterior = arch_corr_ant
cartera_act = corr_act

# base_actual=arch_menr_act
# base_anterior=arch_menr_ant
# cartera_act=men_act

# base_actual=arch_conv_act
# base_anterior=arch_conv_ant
# cartera_act=conv_act



base_anterior['new']='si'
multa_anterior=base_anterior[['NUMERO_CUENTA','new']]
ced_anteriores=base_anterior[['IDENTIFICACION_DEUDOR','new']]

### CRUCE PARA COMPARAR

In [9]:
cruce_multas=pd.merge(base_actual,multa_anterior,on='NUMERO_CUENTA',how='left')

### FILTRO POR VALORES NULOS

In [10]:
nuevas_multas=cruce_multas[cruce_multas['new'].isnull()]

In [11]:
nuevas_multas = nuevas_multas.fillna('')

In [12]:
#subir multas
contador_subidos=0
for row in base_actual.itertuples():
    insert_querry="""INSERT INTO `SISTEMECUADOR_ATM`.`MULTAS_MENSUALES` 
                        (`Cedula`, `No_multa`, `Anio`, `Mes`, `Dia`, 
                         `Dias_mora`, `Saldo`, `Cartera`, `Segmento`, 
                         `Descripcion`, `Placa`) 
                  VALUES('{0}', '{1}', '{2}', '{3}', 
                          '{4}', '{5}', '{6}', '{7}', '{8}',
                          '{9}', '{10}');
    """.format(row.IDENTIFICACION_DEUDOR , row.NUMERO_CUENTA , Anio_act , Mes_act ,
                 Dia_act ,row.DIAS_MORA , row.SALDO , cartera_act['cartera'] , row.SUBCAMPANAPORCLIENTE ,
                row.DESCRIPCION_INFRACION,row.PLACA_VEHICULO)
    try:
        insert=cur.execute(insert_querry)
        db.commit()
        contador_subidos=contador_subidos+1
    except:
        db.rollback()
    
print("CANTIDAD DE MULTAS QUE FUERON SUBIDOS : "+str(contador_subidos))

CANTIDAD DE MULTAS QUE FUERON SUBIDOS : 63


In [13]:
info_subidos=0
contador_subidos=0
for row in base_actual.itertuples():
    CEDULA=row.IDENTIFICACION_DEUDOR
    if len(CEDULA)==9 or len(CEDULA)==12 :
        if CEDULA.isdigit():
            CEDULA='0'+CEDULA
    info_querry="""INSERT INTO `SISTEMECUADOR_ATM`.`INFORMACION_CLIENTE` 
                    (`CEDULA`, `TIPO_IDENTIFICACION`, `RAZON_SOCIAL`,`PRIMER_APELLIDO`, 
                     `SEGUNDO_APELLIDO`, `PRIMER_NOMBRE`, `SEGUNDO_NOMBRE`) 
                    VALUES ('{0}', '{1}', '{2}', '{3}', '{4}', '{5}','{6}');
                """.format(CEDULA,row.TIPO_IDENTIFICACION,row.RAZONSOCIAL,
                           row.PRIMER_APELLIDO,row.SEGUNDO_APELLIDO,row.PRIMER_NOMBRE,
                          row.SEGUNDO_NOMBRE)
    try:
        insert=cur.execute(info_querry)
        db.commit()
        info_subidos=info_subidos+1
    except:
        db.rollback()
        
    insert_querry_DB="""INSERT INTO `SISTEMECUADOR_ATM`.`MULTAS` 
                            (`NUMERO_CUENTA`, `ID_CEDULA`, 
                            `SALDO`, `DIAS_MORA`)
                            VALUES ('{0}', '{1}', 
                            '{2}', '{3}');
    """.format(row.NUMERO_CUENTA,CEDULA,row.SALDO,row.DIAS_MORA )
    try:
        insert=cur.execute(insert_querry_DB)
        db.commit()
        contador_subidos=contador_subidos+1
    except:
        db.rollback()
        
    
print("CANTIDAD DE INFRACTORES QUE FUERON SUBIDOS : "+str(info_subidos))
print("CANTIDAD DE MULTAS QUE FUERON SUBIDOS : "+str(contador_subidos))

CANTIDAD DE INFRACTORES QUE FUERON SUBIDOS : 18
CANTIDAD DE MULTAS QUE FUERON SUBIDOS : 63


In [14]:
# INFRACTORES NUEVOS|1|

In [15]:
cruce_cedulas=pd.merge(base_actual,ced_anteriores,on='IDENTIFICACION_DEUDOR',how='left')
nuevas_cedulas=cruce_cedulas[cruce_cedulas['new'].isnull()]
nuevas_cedulas= nuevas_cedulas.fillna('')

In [16]:
nuevo_archivo='NUEVAS CEDULAS DICCIEMBRE CORRIENTE.xlsx'
writer = pd.ExcelWriter(nuevo_archivo, engine='xlsxwriter')
nuevas_cedulas.to_excel(writer, sheet_name='GESTION')
worksheet = writer.sheets['GESTION']
writer.save()

In [17]:
# base_actual = arch_corr_act
# base_anterior = arch_corr_ant
# cartera_act = corr_act

base_actual=arch_menr_act
base_anterior=arch_menr_ant
cartera_act=men_act

# base_actual=arch_conv_act
# base_anterior=arch_conv_ant
# cartera_act=conv_act



base_anterior['new']='si'
multa_anterior=base_anterior[['NUMERO_CUENTA','new']]
ced_anteriores=base_anterior[['IDENTIFICACION_DEUDOR','new']]

In [18]:
cruce_multas=pd.merge(base_actual,multa_anterior,on='NUMERO_CUENTA',how='left')

In [19]:
nuevas_multas=cruce_multas[cruce_multas['new'].isnull()]

In [20]:
nuevas_multas = nuevas_multas.fillna('')

In [21]:
#subir multas
contador_subidos=0
for row in base_actual.itertuples():
    insert_querry="""INSERT INTO `SISTEMECUADOR_ATM`.`MULTAS_MENSUALES` 
                        (`Cedula`, `No_multa`, `Anio`, `Mes`, `Dia`, 
                         `Dias_mora`, `Saldo`, `Cartera`, `Segmento`, 
                         `Descripcion`, `Placa`) 
                  VALUES('{0}', '{1}', '{2}', '{3}', 
                          '{4}', '{5}', '{6}', '{7}', '{8}',
                          '{9}', '{10}');
    """.format(row.IDENTIFICACION_DEUDOR , row.NUMERO_CUENTA , Anio_act , Mes_act ,
                 Dia_act ,row.DIAS_MORA , row.SALDO , cartera_act['cartera'] , row.SUBCAMPANAPORCLIENTE ,
                row.DESCRIPCION_INFRACION,row.PLACA_VEHICULO)
    try:
        insert=cur.execute(insert_querry)
        db.commit()
        contador_subidos=contador_subidos+1
    except:
        db.rollback()
    
print("CANTIDAD DE MULTAS QUE FUERON SUBIDOS : "+str(contador_subidos))

CANTIDAD DE MULTAS QUE FUERON SUBIDOS : 15


In [22]:
info_subidos=0
contador_subidos=0
for row in base_actual.itertuples():
    CEDULA=row.IDENTIFICACION_DEUDOR
    if len(CEDULA)==9 or len(CEDULA)==12 :
        if CEDULA.isdigit():
            CEDULA='0'+CEDULA
    info_querry="""INSERT INTO `SISTEMECUADOR_ATM`.`INFORMACION_CLIENTE` 
                    (`CEDULA`, `TIPO_IDENTIFICACION`, `RAZON_SOCIAL`,`PRIMER_APELLIDO`, 
                     `SEGUNDO_APELLIDO`, `PRIMER_NOMBRE`, `SEGUNDO_NOMBRE`) 
                    VALUES ('{0}', '{1}', '{2}', '{3}', '{4}', '{5}','{6}');
                """.format(CEDULA,row.TIPO_IDENTIFICACION,row.RAZONSOCIAL,
                           row.PRIMER_APELLIDO,row.SEGUNDO_APELLIDO,row.PRIMER_NOMBRE,
                          row.SEGUNDO_NOMBRE)
    try:
        insert=cur.execute(info_querry)
        db.commit()
        info_subidos=info_subidos+1
    except:
        db.rollback()
        
    insert_querry_DB="""INSERT INTO `SISTEMECUADOR_ATM`.`MULTAS` 
                            (`NUMERO_CUENTA`, `ID_CEDULA`, 
                            `SALDO`, `DIAS_MORA`)
                            VALUES ('{0}', '{1}', 
                            '{2}', '{3}');
    """.format(row.NUMERO_CUENTA,CEDULA,row.SALDO,row.DIAS_MORA )
    try:
        insert=cur.execute(insert_querry_DB)
        db.commit()
        contador_subidos=contador_subidos+1
    except:
        db.rollback()
        
    
print("CANTIDAD DE INFRACTORES QUE FUERON SUBIDOS : "+str(info_subidos))
print("CANTIDAD DE MULTAS QUE FUERON SUBIDOS : "+str(contador_subidos))

CANTIDAD DE INFRACTORES QUE FUERON SUBIDOS : 0
CANTIDAD DE MULTAS QUE FUERON SUBIDOS : 15


In [23]:
cruce_cedulas=pd.merge(base_actual,ced_anteriores,on='IDENTIFICACION_DEUDOR',how='left')
nuevas_cedulas=cruce_cedulas[cruce_cedulas['new'].isnull()]
nuevas_cedulas= nuevas_cedulas.fillna('')

In [24]:
nuevo_archivo='NUEVAS CEDULAS DICCIEMBRE MENORES.xlsx'
writer = pd.ExcelWriter(nuevo_archivo, engine='xlsxwriter')
nuevas_cedulas.to_excel(writer, sheet_name='GESTION')
worksheet = writer.sheets['GESTION']
writer.save()

In [25]:
# DEMOGRAFICOS NUEVOS

In [26]:
arch_dem_act=pd.read_excel(open(dem_act['ruta'],'rb'), sheet_name=dem_act['hoja'])
arch_dem_ant=pd.read_excel(open(dem_ant['ruta'],'rb'), sheet_name=dem_ant['hoja'])

In [27]:
arch_dem_act=arch_dem_act[['IDENTIFICACION_DEUDOR','TELEFONO']]
arch_dem_ant=arch_dem_ant[['IDENTIFICACION_DEUDOR','TELEFONO']]

In [28]:
arch_dem_ant['new']='si'
cruce_demografico=pd.merge(arch_dem_act,arch_dem_ant,on=['IDENTIFICACION_DEUDOR','TELEFONO'],how='left')
nuevas_demograficos=cruce_demografico[cruce_demografico['new'].isnull()]
nuevas_demograficos= nuevas_demograficos.fillna('')
nuevas_demograficos

,IDENTIFICACION_DEUDOR,TELEFONO,new
1,100676667,42566560,
26,101048346,74199934,
44,101479673,42029842,
48,101704013,72815758,
51,102229119,42899569,
54,102418142,72400488,
59,102456084,72962777,
60,102456084,72849227,
63,102456084,42849289,
64,102456084,72849227,


In [29]:
nuevo_archivo='NUEVOS DEMOGRAFICOS DICCIEMBRE.xlsx'
writer = pd.ExcelWriter(nuevo_archivo, engine='xlsxwriter')
nuevas_demograficos.to_excel(writer, sheet_name='DEMOGRAFICOS')
worksheet = writer.sheets['DEMOGRAFICOS']
writer.save()